In [8]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pandas as pd

In [10]:
from pokemon_card_generator.data import card_data

cards_df = card_data.get_card_data()

In [11]:
pokemon_df = cards_df[cards_df.supertype == 'Pokémon']

In [12]:
pokemon_df.head()

,id,name,supertype,subtypes,level,hp,types,evolvesFrom,abilities,attacks,...,nationalPokedexNumbers,legalities,images,tcgplayer,evolvesTo,resistances,rules,index,ancientTrait,releaseDate
0,base1-1,Alakazam,Pokémon,[Stage 2],42,80.0,Psychic,Kadabra,"[{'name': 'Damage Swap', 'text': 'As often as ...","[{'name': 'Confuse Ray', 'cost': ['Psychic', '...",...,[65],{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/base1/...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,NaN,NaN,NaN,NaN,NaN,1999-01-09
1,base1-2,Blastoise,Pokémon,[Stage 2],52,100.0,Water,Wartortle,"[{'name': 'Rain Dance', 'text': 'As often as y...","[{'name': 'Hydro Pump', 'cost': ['Water', 'Wat...",...,[9],{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/base1/...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,NaN,NaN,NaN,NaN,NaN,1999-01-09
2,base1-3,Chansey,Pokémon,[Basic],55,120.0,Colorless,NaN,NaN,"[{'name': 'Scrunch', 'cost': ['Colorless', 'Co...",...,[113],{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/base1/...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,[Blissey],Psychic,NaN,NaN,NaN,1999-01-09
3,base1-4,Charizard,Pokémon,[Stage 2],76,120.0,Fire,Charmeleon,"[{'name': 'Energy Burn', 'text': 'As often as ...","[{'name': 'Fire Spin', 'cost': ['Fire', 'Fire'...",...,[6],{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/base1/...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,NaN,Fighting,NaN,NaN,NaN,1999-01-09
4,base1-5,Clefairy,Pokémon,[Basic],14,40.0,Colorless,NaN,NaN,"[{'name': 'Sing', 'cost': ['Colorless'], 'conv...",...,[35],{'unlimited': 'Legal'},{'small': 'https://images.pokemontcg.io/base1/...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,[Clefable],Psychic,NaN,NaN,NaN,1999-01-09


In [13]:
time_series_df = pokemon_df[['name', 'releaseDate', 'hp' ]]

In [7]:
time_series_df

,name,releaseDate,hp
0,Alakazam,1999-01-09,80.0
1,Blastoise,1999-01-09,100.0
2,Chansey,1999-01-09,120.0
3,Charizard,1999-01-09,120.0
4,Clefairy,1999-01-09,40.0
...,...,...,...
13391,Single Strike Urshifu VMAX,2021-03-19,330.0
13392,Rapid Strike Urshifu VMAX,2021-03-19,330.0
13393,Corviknight VMAX,2021-03-19,320.0
13400,Octillery,2021-03-19,110.0
